![Logo](img/logo.jpg)

# RAPPORT DU PROJET ULTIMATE TIC TAC TOE 2017-2018 Sorbonne Université
## Abdelraouf KESKES  3701191  


### 1) Introduction
Dans le cadre de l’UE 3I025, J'ai étudié plusieurs problématiques dans la théorie des jeux notamment sur des systèmes multi-agents , et comme application j'ai pris le problème du ' Ultimate Tic Tac Toe ' dans le cadre d'un jeu autonome de deux joueurs .Ce dernier a été basé particulièrement sur l'algorithme A\* pour tous les déplacements des joueurs , on verra plus de détails dans la suite .
### 2) Présentation du jeu et des règles 
Dans ce jeu , on met en compétition deux joueurs qui s'affrontent sur une grille contenant  9 grilles du fameux jeux tic tac toe 3 x 3  
Les règles de base sont les suivantes:  
* chaque joueur, à son tour, ramasse une fiole de sa couleur et l'utilise comme un pion pour jouer sur un petit terrain de TicTacToe
* en jouant sur une case donnée de son terrain, le joueur contraint le terrain sur lequel l'autre joueur pourra jouer. 
* lorsqu'un joueur gagne une partie sur un petit terrain, il gagne ce terrain
* Si un petit terrain est gagné par l'un des deux joueurs il devient plus jouable 
* lorsqu'un joueur aligne trois petits terrains victorieux, il gagne globalement la partie


### 3) L'Algorithme A\* et son implémentation
Pour l'implémenter tout d'abord j'ai eu besoin d'un tas à priorité  pour gérer la frontière

In [4]:
import heapq
############### PriorityQueue Class pour gérer la frontière ########################

class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]

#####################################################################################

Ensuite comme heuristique de l'algorithme A* , J'ai utilisé la distance de manhattan : 

In [5]:
#Distance de manhattan 
def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

Et enfin J'ai implémenté  l'algorithme A* par la fonction  __a_star_search__ qui prend en argument notre grille de jeu (graph) , avec une case de départ et une case d'arrivée et il retourne : 
* une liste des came_from ( le Noeud Parent Optimal )  de toutes les cases exploitées
* une liste des coûts de toutes les cases ( Les noeuds ) exploitées

In [7]:
def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

### 4) Classes  et Fonctions  utilisées 
##### Classe GameGrid :
C'est une classe représentant la grille du jeu définie par sa longueur (*width*) , largeur (*height* )  , ses murs ( *walls* ) et un attribut poids ( *weights* ) qui nous aidera pour l'application de l'algorithme A\* . 
Ses méthodes : 
* in_bounds : elle prend une case en argument (x,y) et retourne vrai  si c'est une case dans la grille du jeu
* passable  : elle prend une case en argument (x,y) et retourne Vrai si c'est un point qui n'est pas mur
* neighbors : elle prend une case en argument (x,y) et retourne la liste de ses points voisins valides ( qui sont ni mur ni en dehors la grille ) 
* reconstruct_path : elle prend en argument une liste (came_from ) [ qui pour chaque case (x,y) nous donne son père la case  par laquelle on a pu l'atteindre ] , deux cases  , une case de départ et une case d'arrivée .Elle retourne le chemin optimal  entres ses deux cases en partant bien évidemment de la case de départ  et en appliquant A* .
* Cost : Elle retourne le coût entre deux cases   et dans notre exemple de ce jeu le coût sera 1 entre deux cases voisines .

In [ ]:
class GameGrid:
    def __init__(self, width, height,walls):
        self.width = width
        self.height = height
        self.walls = walls
        self.weights = {}
    
    #Si un point id=(x,y)  est une case dans le terrain du jeu
    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    #Si une Case id=(x,y)  n'est pas une Case Mur
    def passable(self, id):
        return id not in self.walls

    #Retourne tous les voisins valides d'une case id
    def neighbors(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        if (x + y) % 2 == 0: results.reverse() # aesthetics
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return results

    #Retourne le chemin optimal d'une case 'start' vers une case 'goal'
    #Elle sera appelée directement après L'execution de l algorithme A*
    def reconstruct_path(self,came_from, start, goal):
        current = goal
        path = []
        while current != start:
            path.append(current)
            current = came_from[current]
        path.append(start) # optional
        path.reverse() # optional
        return path

    #Retourne le coût d'un noeud source vers Noeud Destinataire
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)

##### La fonction init :
Une fonction prédéfinie par l'énnoncè du projet , permettant d'initialiser la partie graphique du Jeu :

In [8]:
init(_boardname=None):
    global player,game
    # pathfindingWorld_MultiPlayer4
    name = _boardname if _boardname is not None else 'tictactoeBis'
    game = Game('Cartes/' + name + '.json', SpriteBuilder)
    game.O = Ontology(True, 'SpriteSheet-32x32/tiny_spritesheet_ontology.csv')
    game.populate_sprite_names(game.O)
    game.fps = 20  # frames per second
    game.mainiteration()
    game.mask.allow_overlaping_players = True
    #player = game.player

SyntaxError: invalid syntax (<ipython-input-8-c92dc08c0a09>, line 1)

##### La fonction couleur : 
Elle prend en argument un objet du Jeu et retourne sa couleur  ( j : Jaune , B : Bleu ) 

In [ ]:
def couleur(o):
        if o.tileid==tile_fiole_jaune:
            return 'j'
        elif o.tileid==tile_fiole_bleue:
            return 'b'

#### La fonction  put_next_fiole :
* elle prend en argument un joueur et un tour pour lui générer une fiole et la mettre aléatoirement dans le contour de la grille  des 9 petits terrains (3x3)
- Elle retourne un tuple de la position générée (x,y)

In [ ]:
 def put_next_fiole(j,t):
        o = all_fioles[j][t]
    
        # et on met cette fiole qqpart au hasard
    
        x = random.randint(1,19)
        y = random.randint(1,19)
    
        while (x,y) in tictactoeStates or (x,y) in wallStates: # ... mais pas sur un mur
            x = random.randint(1,19)
            y = random.randint(1,19)
        o.set_rowcol(x,y)
        # on ajoute cette fiole dans le dictionnaire
        #fioles[(x,y)]=couleur(o)
    
        game.layers['ramassable'].add(o)
        game.mainiteration()
        return (x,y)

#### La fonction win_index :
Elle prend en argument une grille 3x3 et une couleur , elle cherche un indice gagnant pour une couleur par rapport à l'état actuel de la grille  le premier indice trouvé elle le retourne , elle le retourne .

In [ ]:
#retourne l indexe d'une case gagnante 
def win_index ( Grille , color ) :
        for tuplee in ligne_gagnante :
            count = 0
            winindex = 0
            for i in range(3):
                if ( Grille[tuplee[i]][2] != '' and Grille[tuplee[i]][2] == color ) :
                    count += 1
                else :
                    winindex = i 
            if ( count == 2 ) :
                return winindex

#### La fonction    is_Win :
qui prend en argument un joueur ( indice ) et dit si ce dernier a gagné la partie ou pas 

In [ ]:
def is_Win(joueur):
        for tuplee in ligne_gagnante :
            counter = 0 
            for el in tuplee :
                if ( joueur == 1 ) : grillesGagnee = grille_gagneeJ1
                else : grillesGagnee = grille_gagneeJ2
                if ( el in grillesGagnee ) : counter += 1
            if ( counter == 3 ) :
                print (" Le joueur ",joueur," a gagné")
                sys.exit ()

#### La fonction dead_grid :
Elle prend en argument l'indice d'un petit terrain 3x3 et retourne vrai si ce terrain est gagné .

Elle teste aussi  si un joueur a gagné la partie entière ou pas dans le cas ou un petit terrain a été gagné par un des deux joueurs

In [ ]:
def dead_grid (index) :
        Grille = Grilles[index]
        for tuplee in ligne_gagnante :
            count1 = 0
            count2 = 0
            for i in range(3):
                if ( Grille[tuplee[i]][2] != '' and Grille[tuplee[i]][2] == 'j' ) :
                    count1 += 1
                if ( Grille[tuplee[i]][2] != ''and Grille[tuplee[i]][2] == 'b' ) :
                    count2 += 1
            if ( count1 == 3 ) :
                    grille_gagneeJ1.add(index)
                    is_Win(1)
                    return True 
            if ( count2 == 3 ) :
                    grille_gagneeJ2.add(index)
                    is_Win(2)
                    return True 

### 5) Détails Techniques
+ La terrain global est représenté par une une liste de 9 listes  , chacunes de ces dernières  représente un terrain 3x3  et pour chaque  case je sauvgarde ses 2 indices (x,y) - pour agir sur elle graphiquement -  et son contenu soit '' soit 'j' ou 'b' .
+ __Stratégie Joueur 1__ : Déjà il joue toujours en premier , ça augmente ses chances pour gagner , la stratégie que j'ai choisie repose essentiellement  sur plusieurs parties que j'ai joué Online contre un bot et je l'ai battu en plusieurs reprises  avec une stratégie simple et efficace que j'ai développée , j'ai pas pu l'implémenter en entier  car des fois ça nécessite de l'intelligence humaine qui peut être implémentée avec des tests  plus approfondis or j'ai pas eu assez de temps   pour le faire vu le DeadLine . La stratégie dit qu'il faut jouer  dans les coins des grilles 3 x 3 en commençant par la table du milieu de préférence  les cases de coin sont 0 , 2 , 6 , 8 . Le premier terrain qu'on vise de gagner est le terrain  milieu et le plutôt possible et empêcher l'adversaire de le gagner . Au début , l'adversaire  va gagner  quelques petits terrains de coins .Ensuite la situation se renverse et il se retrouve coincé . Le moindre geste qu'il joue ça nous fait gagner un petit terrain .
J'ai gagné plus de 40 parties avec cette stratégie 

Voici quelques exemples : 
![alt-text-1](img/win1.png) ![alt-text-2](img/win2.png) ![alt-text-3](img/win3.png)

+ __Stratégie Joueur 2__ : J'ai fait en sorte qu'il joue aléatoirement  parce que si je mets  deux joueurs stratégiques dans la majorité des cas  ça finira toujours par une égalité , et c'est pas ce qu'on cherche vraiment à obtenir ,et comme j'ai pas implémenté entièrement la stratégie du joueur 1 y'aura des cas ou le joueur aléatoire gagnera,ça reste probable 

### 6) Conclusion
D'après de longues recherches sur internet  et de nombreuses parties joué , j'ai réalisé que ce jeu reste quand même très stratégique et qu'on est toujours pas arrivé à une stratégie toujours gagnante , même avec le reinforcement learning , le Bot reste quand même battable si on joue en premier . 

### Références
https://mathwithbaddrawings.com/ultimate-tic-tac-toe-original-post/

https://en.wikipedia.org/wiki/Ultimate_tic-tac-toe

https://boardgames.stackexchange.com/questions/12477/is-ultimate-tic-tac-toe-solved

http://www.cs.huji.ac.il/~ai/projects/2013/U2T3P/files/AI_Report.pdf

http://www.redblobgames.com/pathfinding/a-star/introduction.html